<a href="https://colab.research.google.com/github/jingyuan0422/113-2-Programing-Language/blob/main/PL_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [190]:
# Authorize Google Drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [274]:
# Output from GoogleSheet "thoughts"
import pandas as pd
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1EXd_yATBW5z3S6l4yIAnU2-toARKm3zgidVChcNyeyg/edit?gid=0#gid=0')

In [275]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1EXd_yATBW5z3S6l4yIAnU2-toARKm3zgidVChcNyeyg/edit?gid=0#gid=0').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [276]:
from gspread_dataframe import set_with_dataframe

# Select the first worksheet
worksheet = gsheets.get_worksheet(0)

# return a 2D array
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])

# 去除 contnet 欄位中的空行
df = df[df['content'].str.strip() != '']

In [277]:
df

,content
0,西藏的旅程，充滿太多不可思議，也突破自己很多以為的不可能，
2,能平安回來，先感謝老天爺、謝謝導遊、團員、夥伴們的互相照顧。
4,這趟出遊，
6,從9/20出發，10/2回到台灣(2013年)，有整整十天，我待在西藏，
8,再另外三天，我則在搭火車、搭飛機，交通方面的確有辛苦到，
...,...
456,嘿嘿嘿，這一鍋大餐，是四川干鍋，我們點了以鴨翅、排骨為基鍋(是這樣說嗎？)
458,好險，這一天我也可以正常進食，
460,於是乎，我吃得超級無敵開心，
462,"當然，回飯店後就高原反應了，然後咧，我還住在馬桶上@""@"


In [278]:
!pip install jieba
!pip install snownlp

In [279]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [280]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [281]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [282]:
df['Chinese_Content'] = df['content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [283]:
df['Chinese_Content']

,Chinese_Content
0,西藏的旅程充滿太多不可思議也突破自己很多以為的不可能
2,能平安回來先感謝老天爺謝謝導遊團員夥伴們的互相照顧
4,這趟出遊
6,從出發回到台灣年有整整十天我待在西藏
8,再另外三天我則在搭火車搭飛機交通方面的確有辛苦到
...,...
456,嘿嘿嘿這一鍋大餐是四川干鍋我們點了以鴨翅排骨為基鍋是這樣說嗎
458,好險這一天我也可以正常進食
460,於是乎我吃得超級無敵開心
462,當然回飯店後就高原反應了然後咧我還住在馬桶上


In [284]:
df['Tokenized']

,Tokenized
0,"[西藏, 的, 旅程, 充滿, 太多, 不可思議, 也, 突破, 自己, 很多, 以為, 的..."
2,"[能, 平安, 回來, 先, 感謝, 老天, 爺, 謝謝, 導遊, 團員, 夥伴, 們, 的..."
4,"[這趟, 出遊]"
6,"[從, 出發, 回到, 台灣, 年, 有, 整整, 十天, 我待, 在, 西藏]"
8,"[再, 另外, 三天, 我則, 在, 搭, 火車, 搭, 飛機, 交通, 方面, 的, 確有..."
...,...
456,"[嘿嘿嘿, 這一鍋, 大餐, 是, 四川, 干鍋, 我們, 點了, 以, 鴨, 翅, 排骨,..."
458,"[好險, 這, 一天, 我, 也, 可以, 正常, 進食]"
460,"[於, 是, 乎, 我, 吃, 得, 超級, 無敵, 開心]"
462,"[當然, 回飯, 店, 後, 就, 高原, 反應, 了然, 後, 咧, 我, 還住, 在, ..."


In [285]:
word_counts = Counter(all_words)
word_counts

Counter({'西藏': 29,
         '旅程': 2,
         '充滿': 3,
         '太多': 1,
         '不可思議': 5,
         '突破': 1,
         '自己': 15,
         '很多': 1,
         '以為': 2,
         '可能': 4,
         '平安': 3,
         '回來': 2,
         '感謝': 1,
         '老天': 2,
         '謝謝': 2,
         '導遊': 2,
         '團員': 2,
         '夥伴': 1,
         '互相': 2,
         '照顧': 2,
         '這趟': 2,
         '出遊': 1,
         '出發': 2,
         '回到': 1,
         '台灣': 8,
         '整整': 1,
         '十天': 1,
         '我待': 1,
         '另外': 1,
         '三天': 1,
         '我則': 3,
         '火車': 6,
         '飛機': 4,
         '交通': 1,
         '方面': 1,
         '確有': 1,
         '辛苦': 4,
         '人生': 2,
         '第一次': 1,
         '小時': 2,
         '兩天': 1,
         '時間': 1,
         '寶島': 1,
         '多麼': 1,
         '新奇': 1,
         '體驗': 1,
         '神奇': 1,
         '第一天': 1,
         '開始': 2,
         '青藏鐵路': 9,
         '一直': 1,
         '無法': 3,
         '克服': 2,
         '不能': 4,
         '天天': 3,
  

In [286]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set([
    '所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '非常', '真的', '突破', '太多', '很多',
    '感覺', '還是', '我們', '就是', '有點', '因為', '整體', '覺得', '這次', '部分', '但是', '一個', '這個', '沒有', '一樣', '今天', '還是', '我們', '就是',
    '一直', '只有', '雖然', '以為', '不是', '可能', '應該', '不要', '竟然', '超級', '這樣', '另外', '點了', '比較', '一點', '更是', '旁邊', '以往', '唯一',
    '一些', '是否', '除了', '什麼', '那麼', '當然', '依然', '依舊', '多麽', '這趟', '只能', '這家', '算是', '而言', '本身', '才能', '得到', '可不是', '尤其',
    '超級', '以前', '蠻', '過', '有此'
                  ])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [287]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

In [288]:
word_freq_df

,Word,Frequency
0,西藏,29
1,高原,16
2,自己,15
3,反應,14
4,還有,13
...,...,...
875,茶館,1
876,方式,1
877,爆炸,1
878,有趣,1


In [289]:
# 假設 filtered_word_counts 是一個 Counter 或 dict 物件
keywords = list(filtered_word_counts.keys())

prompt = f"""
我有一批中文旅遊遊記的文章，我已經用斷詞工具 jieba 把詞切好了。
請你幫我針對這些詞做命名實體識別 NER 分類，將每個詞分成以下五類之一：

1. 景點（例如：城市、山脈、湖泊、寺廟、觀光區等）
2. 美食（例如：地方小吃、餐廳、飲品等）
3. 交通方式（例如：火車、飛機、巴士、租車等）
4. 心得感想（例如：形容旅程的情緒、體驗或感受、跟主觀感受有關的所有詞）
5. 其他（無法歸類者）

以下是我整理好的詞彙：
{keywords}

整理完後請幫我用以下方式呈現：蘋果 美食、拉薩 景點，並以表格呈現
"""

In [290]:
prompt

"\n我有一批中文旅遊遊記的文章，我已經用斷詞工具 jieba 把詞切好了。\n請你幫我針對這些詞做命名實體識別 NER 分類，將每個詞分成以下五類之一：\n\n1. 景點（例如：城市、山脈、湖泊、寺廟、觀光區等）\n2. 美食（例如：地方小吃、餐廳、飲品等）\n3. 交通方式（例如：火車、飛機、巴士、租車等）\n4. 心得感想（例如：形容旅程的情緒、體驗或感受、跟主觀感受有關的所有詞）\n5. 其他（無法歸類者）\n\n以下是我整理好的詞彙：\n['西藏', '旅程', '充滿', '不可思議', '自己', '平安', '回來', '感謝', '老天', '謝謝', '導遊', '團員', '夥伴', '互相', '照顧', '出遊', '出發', '回到', '台灣', '整整', '十天', '我待', '三天', '我則', '火車', '飛機', '交通', '方面', '確有', '辛苦', '人生', '第一次', '小時', '兩天', '時間', '寶島', '多麼', '新奇', '體驗', '神奇', '第一天', '開始', '青藏鐵路', '無法', '克服', '不能', '天天', '洗澡', '洗頭', '重要', '人生大事', '想不到', '辦到', '呵呵', '怎能', '即便', '幸運地', '搭乘', '班列', '車最豪華', '軟臥', '包廂能', '刷牙', '洗臉', '已經', '幸福', '桃園', '抵達', '香港', '轉機', '成都', '機場', '計程', '車花', '將近', '一小', '兩班', '滿滿', '人山人海', '中終', '取得', '兩張', '火車票', '這火', '車票', '事先', '預訂', '透過', '當地', '旅行社', '這兩張', '可不', '簡單', '人來', '目前', '並無法', '購買', '這是', '西寧', '拉薩', '一段', '可是', '嚴格', '管制', '不但', '之外', '自助', '旅行', '這也', '加上', '必須', '辦理入', '這當然', '可以', '辦理', '一大堆', '邊防證', '人要', '旅遊', '函證', '但別', '緊張', '從頭', '刁難', '每天', 

In [291]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(prompt)
print(response.text)

好的，我將您提供的詞彙進行命名實體識別 (NER) 分類，並以表格方式呈現結果。

| 詞彙     | 分類     |
| -------- | -------- |
| 西藏     | 景點     |
| 旅程     | 心得感想 |
| 充滿     | 心得感想 |
| 不可思議 | 心得感想 |
| 自己     | 其他     |
| 平安     | 心得感想 |
| 回來     | 心得感想 |
| 感謝     | 心得感想 |
| 老天     | 心得感想 |
| 謝謝     | 心得感想 |
| 導遊     | 其他     |
| 團員     | 其他     |
| 夥伴     | 其他     |
| 互相     | 心得感想 |
| 照顧     | 心得感想 |
| 出遊     | 心得感想 |
| 出發     | 心得感想 |
| 回到     | 心得感想 |
| 台灣     | 景點     |
| 整整     | 其他     |
| 十天     | 其他     |
| 我待     | 其他     |
| 三天     | 其他     |
| 我則     | 其他     |
| 火車     | 交通方式 |
| 飛機     | 交通方式 |
| 交通     | 交通方式 |
| 方面     | 其他     |
| 確有     | 心得感想 |
| 辛苦     | 心得感想 |
| 人生     | 心得感想 |
| 第一次   | 其他     |
| 小時     | 其他     |
| 兩天     | 其他     |
| 時間     | 其他     |
| 寶島     | 景點     |
| 多麼     | 心得感想 |
| 新奇     | 心得感想 |
| 體驗     | 心得感想 |
| 神奇     | 心得感想 |
| 第一天   | 其他     |
| 開始     | 心得感想 |
| 青藏鐵路 | 景點 & 交通方式     |
| 無法     | 心得感想 |
| 克服     | 心得感想 |
| 不能     | 心得感想 |
| 天天     | 其他     |
| 洗澡     | 其他     |
| 洗頭   

In [292]:
import pandas as pd

response_text = response.text

# 定義五個分類的列表
sights = []
foods = []
transports = []
feelings = []
others = []

# 分割文本為每一行，並去除每行前後的空格
lines = response_text.strip().split("\n")

# 迭代每一行並根據分類將詞彙存入對應的列表
for line in lines:
    # 去除前後的空格和 '|' 符號，並處理多餘的空格
    cleaned_line = line.strip('|').strip().replace('  ', ' ')  # 用單個空格代替多餘的空格

    # 確保每行有兩個元素（詞彙和分類）
    if " | " in cleaned_line:
        word, category = cleaned_line.split(" | ")

        # 根據分類將詞彙加入對應的列表
        if category == '景點':
            sights.append(word)
        elif category == '美食':
            foods.append(word)
        elif category == '心得感想':
            feelings.append(word)
        elif category == '交通方式':
            transports.append(word)
        else:
            others.append(word)

# 顯示每個分類的結果
print("景點:", sights)
print("美食:", foods)
print("心得感想:", feelings)
print("交通方式:", transports)
print("其他:", others)

景點: ['西藏  ', '台灣  ', '寶島  ', '桃園  ', '香港  ', '成都  ', '當地  ', '西寧  ', '拉薩  ', '雪景  ', '四川  ', '海拔  ', '戶外  ', '景象  ', '高山  ', '布達拉宮', '宮殿  ', '布宮  ', '日喀則 ', '珠峰  ', '聖母  ', '峰山  ', '絨布  ', '古廟  ', '山頂  ', '雍布拉康', '埃及  ', '草原  ', '大自然 ', '聖母峰 ', '雲層  ', '山峰  ', '大本營 ', '山上  ', '平地  ', '大陸  ', '湖納木措', '納木措 ', '水湖  ', '登山  ', '湖畔  ', '星星  ', '胡楊  ', '溫泉  ', '定日  ', '農村  ', '市區  ', '大昭寺 ', '轉經道 ', '卡若拉 ', '冰川  ']
美食: ['餐廳  ', '用餐  ', '川菜  ', '酥油茶 ', '魚的  ', '甜茶  ', '早餐  ', '這張餅 ', '餅吃  ', '吃顆  ', '青稞  ', '酸奶  ']
心得感想: ['旅程  ', '充滿  ', '不可思議', '平安  ', '回來  ', '感謝  ', '老天  ', '謝謝  ', '互相  ', '照顧  ', '出遊  ', '出發  ', '回到  ', '確有  ', '辛苦  ', '人生  ', '多麼  ', '新奇  ', '體驗  ', '神奇  ', '開始  ', '無法  ', '克服  ', '不能  ', '重要  ', '想不到 ', '辦到  ', '呵呵  ', '怎能  ', '幸運地 ', '車最豪華', '已經  ', '幸福  ', '滿滿  ', '人山人海', '可不  ', '簡單  ', '並無法 ', '可是  ', '嚴格  ', '管制  ', '不但  ', '自助  ', '旅行  ', '可以  ', '旅遊  ', '但別  ', '緊張  ', '刁難  ', '費心  ', '如何  ', '行動  ', '並不難 ', '上火  ', '好玩  ', '認真  ', '分享  ', '有無限 ', '敬佩  ', '真是  ',

In [300]:
from itertools import zip_longest

# 組成 df
df_final = pd.DataFrame(
    list(zip_longest(sights, foods, feelings, transports, fillvalue='')),
    columns=['景點', '美食', '心得感想', '交通方式']
)

In [301]:
df_final

,景點,美食,心得感想,交通方式
0,西藏,餐廳,旅程,火車
1,台灣,用餐,充滿,飛機
2,寶島,川菜,不可思議,交通
3,桃園,酥油茶,平安,搭乘
4,香港,魚的,回來,班列
...,...,...,...,...
338,,,好好,
339,,,研究,
340,,,發紫,
341,,,太不容易,


In [302]:
# 去除 df_final 中每個欄位值的前後空格
df_final_cleaned = df_final.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
word_freq_df = word_freq_df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

# 儲存分類結果
types = []

# 對每個 word 進行分類
for word in word_freq_df['Word']:
    word_cleaned = word.strip()  # 去除字詞本身的空白
    if word_cleaned in df_final_cleaned['景點'].values:
        types.append('景點')
    elif word_cleaned in df_final_cleaned['美食'].values:
        types.append('美食')
    elif word_cleaned in df_final_cleaned['交通方式'].values:
        types.append('交通方式')
    elif word_cleaned in df_final_cleaned['心得感想'].values:
        types.append('心得感想')
    else:
        types.append('其他')

# 新增 type 欄位
word_freq_df['type'] = types

In [303]:
word_freq_df

,Word,Frequency,type
0,西藏,29,景點
1,高原,16,其他
2,自己,15,其他
3,反應,14,心得感想
4,還有,13,其他
...,...,...,...
875,茶館,1,其他
876,方式,1,其他
877,爆炸,1,其他
878,有趣,1,其他


In [304]:
# 1. 景點詞頻圖
sights_df = word_freq_df[word_freq_df['type'] == '景點'].sort_values(by='Frequency', ascending=False)
fig1 = px.bar(sights_df.head(30), x='Word', y='Frequency',
             title='景點出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig1.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig1.show()

In [305]:
# 2. 美食方式詞頻圖
foods_df = word_freq_df[word_freq_df['type'] == '美食'].sort_values(by='Frequency', ascending=False)
fig2 = px.bar(foods_df.head(30), x='Word', y='Frequency',
             title='美食出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig2.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig2.show()

In [306]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-wqy-zenhei is already the newest version (0.9.45-8).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [307]:
# 3. 交通方式詞頻圖
transports_df = word_freq_df[word_freq_df['type'] == '交通方式'].sort_values(by='Frequency', ascending=False)
fig3 = px.bar(transports_df.head(30), x='Word', y='Frequency',
             title='交通方式出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig3.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig3.show()

In [310]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
feelings_df = word_freq_df[word_freq_df['type'] == '心得感想'].sort_values(by='Frequency', ascending=False)
top_words = feelings_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig4 = px.imshow(wordcloud_img)
fig4.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig4.show()

In [314]:
prompt = f"""
以下是文章內容，請為其提供總結並給出結論，並提供一些針對有意前往這些地點旅遊的遊客的注意事項或建議：

{df.to_string()}

總結這篇文章，並針對計劃前往這些景點的遊客，提供一些實用的旅遊建議和注意事項：
"""

response_summary = model.generate_content(prompt)
print(response_summary.text)

好的，根據文章內容，以下是總結、結論以及給有意前往西藏旅遊的遊客的建議和注意事項：

**文章總結：**

這篇文章是一位台灣遊客分享她前往西藏旅遊的經歷。旅程主要包括搭乘青藏鐵路前往拉薩，遊覽布達拉宮等著名景點，以及前往日喀則地區的絨布寺和珠峰大本營。作者詳細描述了旅途中的所見所聞，包括壯麗的自然風光、獨特的人文風情，以及高原反應帶來的挑戰。她也分享了搭乘青藏鐵路、購買車票、在西藏的交通方式，以及在高原地區的飲食、住宿等方面的經驗。

**結論：**

西藏之旅充滿了令人難以置信的體驗，但也伴隨著高原反應等挑戰。儘管存在一些限制（如台灣遊客無法自助旅行、需要辦理入藏函等），但透過旅行社的安排，依然可以欣賞到西藏獨特的自然風光和文化。

**給有意前往西藏旅遊的遊客的建議和注意事項：**

*   **行程規劃：**
    *   **提前預訂：** 青藏鐵路車票對台灣遊客有購買限制，且西藏不允許自助旅行，務必提前透過旅行社預訂行程、車票及辦理入藏函。
    *   **交通考量：** 前往西藏的交通方式多樣，包括飛機和火車。火車雖然耗時較長（例如，44 小時），但可以逐漸適應海拔高度。
    *   **行程安排：** 西藏的景點分散，建議規劃至少十天以上的行程，以便充分體驗各地的風光。
*   **健康與安全：**
    *   **高原反應：** 高原反應是西藏旅遊的主要挑戰。建議提前諮詢醫生，準備相關藥物（例如，高原康）。在旅途中注意放慢節奏，避免過度勞累，並隨時補充水分。
    *   **身體狀況：** 出發前評估自己的身體狀況，確保適合前往高海拔地區。
    *   **緊急應對：** 了解高原反應的症狀和應對方法，並與導遊保持密切聯繫。
*   **生活與文化：**
    *   **住宿條件：** 西藏的住宿條件相對簡樸，特別是在偏遠地區（如珠峰大本營）。要有心理準備，可能無法每天洗澡，且住宿環境較為簡陋。
    *   **飲食習慣：** 西藏的飲食以藏餐為主，如糌粑、酥油茶等。可以嘗試當地特色美食，但也要注意飲食衛生，避免腸胃不適。
    *   **尊重文化：** 尊重當地的宗教信仰和風俗習慣。在寺廟等宗教場所，注意言行舉止，並聽從導遊的指示。
*   **其他建議：**
    *   **衣物準備：** 西藏的氣候多變，